In [1]:
from MCFR2 import *
from utils import *

c:\Users\jbval\PDM\01_composition\MCFR.py:165: SyntaxWarning: invalid escape sequence '\l'
  label=f"mflow ${{\lambda_{{in}}}}$={self.list_MCFR[i].mflow_in} /s"
c:\Users\jbval\PDM\01_composition\MCFR.py:549: SyntaxWarning: invalid escape sequence '\l'
  plt.title(f"{variable}(BU) {self.lib_cl}/{self.lib_all} for ${{\lambda_{{in}}}}$={self.mflow_in} /s, Cl_e={self.e_cl*100} w%",fontsize = 10)
c:\Users\jbval\PDM\01_composition\MCFR.py:585: SyntaxWarning: invalid escape sequence '\l'
  plt.title(f'Evolution of {variable} for ${{\lambda_{{in}}}}$={self.mflow_in} /s, Cl_e={self.e_cl*100} w%',fontsize = 10)


The template folder includes all the files required to generate the serpent files, as well as the execute file for the Savio cluster. 

It also includes the properties.json file, which includes all the main properties which are not usually changed for each model, such as salt temperature, library path, dimensions of the core and so on. 

In [ ]:
###################################    STATIC    ###########################################
# The first class of interest is the Static class which allows to run static calculation

#this initiates an instance of the class which will store all the variables necessary to generate the serpent file, it will also create a directory name based on the 
#the variables selected, this will also write in this directory the serpent file.
static=Static(e_cl=0.75,lib_cl="LANL",reactor_type="MCFR_C",get_flux=True,pop=1e6)

#If using the savio cluster then you can run the simulation directly with, you can give it a number of nodes, time in hours and partition
static.run_serpent(nodes=2, time=5,partitions="savio4_htc")

#Once the simulation is over one can extract the results file, this can be done multiple times without rerunning the simulation 

keff=static.extract_res_m("absKeff") #extract the keff

static.extract_flux() #this will plot the flux


In [ ]:

###################################   STATICS    ########################################
#This class allows for a set of multiple Static instances to be put in a list an chained run changing only the cl enrichment
cl_e=np.linspace(0.25,0.99,5)
#generate the class of class empty
statics=Statics()

#add each element of the list to the class in a loop, this will create a single directory with all the different serpent inputs
for i in cl_e:
    statics.add_MCFR(Static(e_cl=i))
    
#Then you can run them all using a single execute file
statics.run_all()

#All the resfiles are read and extracted together here plots the keff in function of Cl_e this can be used to find a critical Cl enrichment
keff=statics.extract_plot("absKeff")


***BURNUP***

Now that a critical Cl enrichment is found then one can look at **burnup calculations**. Here the steps are a bit more complicated due to the necessity of finding the mflow out for each corresponding mflow in, they need to conserve the volume of salt. These mflow represent time constants for the inflow of fresh salt, outflow of salt and of gas.

If the mflow_out is not known then it is found by running a pre-simulation. This pre-simulation for two known values will find the gas mass flow rate, in the case of the MCFR_C an example of a known pair is mflow_in=2.55e-10 and mflow_out=2.467e-10, and MCFR_D: mflow_in=2.55e-10, mflow_out=2.381e-10. If you change the reactor from the base case you need to find a new value of mflow_out-mflow_in which conserves mass and then enter these into the ref_mflow_in and out.  

Now once these reference values are set you can run the simulation for any value of mflow_in. If the value of mflow_out is not set it will run a quick simulation with the reference value to find the gas mass flow rate for the reference case. The gas mass flow rate should not differ drastically from the reference case to your case so it is used for the mass conservation equation in order to obtain mflow_out from your mflow_in. The code will print in the terminal the correct value of mflow_out for you to enter at the next simulation. Note once this pre-simulation is run once the dep.m file will stay in your directory and won't be run again in order to get the mflow_out.

In [ ]:
#################################   Depletion   ################################

#initialise Depletion, if mflow_out not know will run pre-simulation as mentioned above to find mflow_out in the same directory. 
dep=Depletion(mflow_in=3e-10,reactor_type="MCFR_C",BU_years=400)

#if using savio cluster can run it incode
dep.run_serpent()

#extract res.m file, here extract the keff evolution in burnup step
keff=dep.extract_res_m("absKeff",do_plot=True)

#extract dep.m file, select isotopes in a list, select variable to extract "adens", or "mdens" (atomic or mass density)
comp,BU=dep.extract_dep_m(["Pu239","U235","Cl35"],"adens")

In [ ]:
#################################   Depletions   ################################

#Here it is the same idea as Statics but for the Depletion being a class of class instances, however here we itterate on the mflow_in parameter in order to find the equilibrium fuel cycle

deps=Depletions()
mflows=np.linspace(1e-10,1e-9,5)
for i in mflows:
    deps.add_MCFR(Depletion(mflows))

# ... same idea as for Statics

***SENSITIVITY***

The final an most important step is the sensitivity analysis which involves the last class of interest Sensitivity. This class generates the input file for sensitivity calculations given a number of isotopes to perturb and reactions to perturb. It also has two different scenario an initial comp sensitivity and equilibrium composition sensitivity. The sensitivity is determined by the number of latent generations which is set in the template file to 15.

**Initial comp** sensitivity is straight forward it will do the sensitivity based on the given isotopes. The choice of MT can be given in a list or just as all_MT to select all the relevant ones, note however the number of MT entered in the SERPENT input doesn't correspond to the ones outputed since sum reactions such as MT=4 are made up of MT=52-91. Note the number of inactive cycles should be screened in such a way that the number of inactive= #to converge + # latent gen + buffer (i.e. 5).

**Equilibrium comp** In order to do this set equi_comp=True, but also need to set mflows. Sensitivity will at first run a burnup calcluation to reach the equilibrium composition by generating an instance of the burnup class and will write a restart file from which the sensitivity calculation will be done on the last step. The choice of isotopes to perturb for the sens calculation can be done manually by giving some isotopes, or by giving a number which will define how many of the most abundant isotopes will be selected, or simply with "all" which will select the default of the top 70 isotopes. Note that this depletion run will generate the restart file from which as many sensitivity calculations using the same model parameters can be run as the restart file is note wiped.

**Covariance** The covariance data needs to be generated and made available to the script with the path to each library specified in the properties.json file. These files should be in the following format zaid-MT1-zaid-MT2.npy, where zaid is the number format of the isotopes such as Cl35 being 170350 (with a 0 at the end for the metastable state). These are relative covariance which are read and extracted and put together in to the covariance matrix.

**PSD** note there exists a utils file which has many functions inside, the main ones of interest for PSD are: isPD (which checks if smth is PSD), deflate_sparse_matrix_eigen (which returns the PSD corrected matrix).

In [ ]:
######################### initial Sensitivity #################################

#initiallise sens for initial composition, here all means all the isotopes present in the model (24 in our case) will be perturbed
sens=Sensitivity(sens_iso="all",sens_MT="all_MT",equi_comp=False,reactor_type="MCFR_C")

#if using savio cluster, note the memory usage is on the order of 150 GB of memory required (for 15 latent gen, 1e6 neutrons, 24 istopopes and 55 MT), so select partitions that can handle that
sens.run_serpent(time=70)

#The first step of the post is extracting the sensitivity profiles and integral sensitivity from the heavy output file, here the default option is used which extracts the main MT numbers and all the isotopes,
sens.extract_sens_m(zais="all") 
#One could for example also do: sens.extract_sens_m(zais=[922380,922350,170350]) to extract and play around with only these istopes

#once these are extracted they will stay in memory and you can play around with them with for example the option of ranking the integral sensitivities
sens.rank_sens() #if not isotope or reaction is specified will create a plot and a csv file of the rank
sens.rank_sens(isotope=[170350]) # will create a rank for only Cl35 reactions

#to find out which isotope keff is most sensitive to you can rank by the absolute sum of each reactions for each isotope (since a strongly negative and positive reaction would
# both contribute to uncertainty)

sens.abs_contribution_iso()

# Finally to do error propagation, the limit is only necessary when looking at equilbrium fuel cycle, if set to true and the number of reaction-isotope pairs
#exceeds 250 then only these first 250 most sensitive reaction pairs are kept for the error propagation. This accelerates the matrix multiplication. 
sens.error_prop(limit=False)

#This will result in a bar plot with the uncertainty for each isotope and further broken down into perreaction uncertainty. This will also be outputed into a csv file
#with one file for the per isotope uncertainty along with other information about PSDness, and an other file with a per-reaction uncertainty breakdown.

#All this can be repeated by setting equi_comp=True, in theory you c
sensdep=Sensitivity(sens_iso="all",sens_MT="all_MT",equi_comp=True,reactor_type="MCFR_C",mflow_in=...,mflow_out=...)

#... same as before
